<a href="https://colab.research.google.com/github/aekanun2020/2025-AdvancedRAG/blob/main/v3_prove_one_neuronRNN_part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch.nn import Parameter
from torch.autograd import Variable
import torch.nn.functional as F
import math

# กำหนดสีสำหรับแสดงผล
class Colors:
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    RED = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

# กำหนด seed เพื่อให้ผลลัพธ์คงที่
torch.manual_seed(9)

# สร้าง RNN Cell โดยมี input_size=1, hidden_size=1, และไม่มี bias
rnn_a = torch.nn.RNNCell(input_size=1, hidden_size=1, bias=False)

# แสดงค่า parameters ทั้งหมดใน RNN Cell พร้อมคำอธิบาย
print(f"{Colors.BOLD}Parameters ของ RNN Cell:{Colors.ENDC}")
weights = list(rnn_a.parameters())
w_ih = weights[0]  # Input-to-Hidden (Weight W)
w_hh = weights[1]  # Hidden-to-Hidden (Weight U)
print(f"Weight W (input-to-hidden): {w_ih.item():.4f} - ใช้คำนวณอิทธิพลของ input ปัจจุบัน")
print(f"Weight U (hidden-to-hidden): {w_hh.item():.4f} - ใช้คำนวณอิทธิพลของ hidden state ก่อนหน้า")
print(f"\n{Colors.BOLD}สมการของ RNN Cell: h_t = tanh(W·x_t + U·h_{{t-1}}){Colors.ENDC}")
print("  โดย:")
print("    - h_t คือ hidden state ณ เวลา t")
print("    - x_t คือ input ณ เวลา t")
print("    - h_{t-1} คือ hidden state ก่อนหน้า")
print("    - W และ U คือ weight matrices ที่ใช้ในทุก time step (weight sharing)")

# สร้าง input sequence และ hidden state เริ่มต้น
# ค่า input ที่ time step ที่ 1, 2, และ 3
random_input = Variable(torch.FloatTensor(1, 3, 1).normal_(), requires_grad=False)
random_input[0, 0, 0] = 1
random_input[0, 1, 0] = -0.7775
random_input[0, 2, 0] = 0.2588
print(f"\n{Colors.BOLD}Input sequence:{Colors.ENDC}")
print(f"X_1 = {random_input[0, 0, 0].item()}")
print(f"X_2 = {random_input[0, 1, 0].item()}")
print(f"X_3 = {random_input[0, 2, 0].item()}")

# ค่า hidden state เริ่มต้น
h0 = Variable(torch.zeros(1, 1), requires_grad=False)
print(f"\n{Colors.BOLD}Initial hidden state (h_0):{Colors.ENDC} {h0.item()}")

# คำนวณ hidden state ที่แต่ละ time step
print(f"\n{Colors.BOLD}การคำนวณ hidden state ที่แต่ละ time step:{Colors.ENDC}")

# Time step 1
print(f"\n{Colors.BOLD}Time step 1:{Colors.ENDC}")
# คำนวณด้วย PyTorch RNN Cell
h1 = rnn_a(random_input[0, 0], h0)
print(f"  {Colors.BLUE}PyTorch RNN Cell → h_1 = {h1.item():.4f}{Colors.ENDC}  (คำนวณโดย RNN Cell ของ PyTorch)")

# คำนวณแบบ manual
w_x1 = w_ih.item() * random_input[0, 0, 0].item()
u_h0 = w_hh.item() * h0.item()
print(f"  {Colors.GREEN}คำนวณแบบ manual:{Colors.ENDC}")
print(f"  W·x_1 = {w_ih.item():.4f} × {random_input[0, 0, 0].item()} = {w_x1:.4f}")
print(f"  U·h_0 = {w_hh.item():.4f} × {h0.item()} = {u_h0:.4f}")
print(f"  {Colors.GREEN}h_1 = tanh(W·x_1 + U·h_0) = tanh({w_x1:.4f} + {u_h0:.4f}) = tanh({w_x1 + u_h0:.4f}) = {h1.item():.4f}{Colors.ENDC}")

# Verify การคำนวณ
diff1 = abs(h1.item() - torch.tanh(torch.tensor(w_x1 + u_h0)).item())
if diff1 < 1e-6:
    print(f"  ✓ การ verify สำเร็จ! ความแตกต่าง: {diff1:.10f}")
else:
    print(f"  {Colors.RED}✗ การ verify ล้มเหลว! ความแตกต่าง: {diff1:.10f}{Colors.ENDC}")

# Time step 2
print(f"\n{Colors.BOLD}Time step 2:{Colors.ENDC}")
# คำนวณด้วย PyTorch RNN Cell
h2 = rnn_a(random_input[0, 1], h1)
print(f"  {Colors.BLUE}PyTorch RNN Cell → h_2 = {h2.item():.4f}{Colors.ENDC}  (คำนวณโดย RNN Cell ของ PyTorch)")

# คำนวณแบบ manual
w_x2 = w_ih.item() * random_input[0, 1, 0].item()
u_h1 = w_hh.item() * h1.item()
print(f"  {Colors.GREEN}คำนวณแบบ manual:{Colors.ENDC}")
print(f"  W·x_2 = {w_ih.item():.4f} × {random_input[0, 1, 0].item()} = {w_x2:.4f}")
print(f"  U·h_1 = {w_hh.item():.4f} × {h1.item()} = {u_h1:.4f}")
print(f"  {Colors.GREEN}h_2 = tanh(W·x_2 + U·h_1) = tanh({w_x2:.4f} + {u_h1:.4f}) = tanh({w_x2 + u_h1:.4f}) = {h2.item():.4f}{Colors.ENDC}")

# Verify การคำนวณ
diff2 = abs(h2.item() - torch.tanh(torch.tensor(w_x2 + u_h1)).item())
if diff2 < 1e-6:
    print(f"  ✓ การ verify สำเร็จ! ความแตกต่าง: {diff2:.10f}")
else:
    print(f"  {Colors.RED}✗ การ verify ล้มเหลว! ความแตกต่าง: {diff2:.10f}{Colors.ENDC}")

# Time step 3
print(f"\n{Colors.BOLD}Time step 3:{Colors.ENDC}")
# คำนวณด้วย PyTorch RNN Cell
h3 = rnn_a(random_input[0, 2], h2)
print(f"  {Colors.BLUE}PyTorch RNN Cell → h_3 = {h3.item():.4f}{Colors.ENDC}  (คำนวณโดย RNN Cell ของ PyTorch)")

# คำนวณแบบ manual
w_x3 = w_ih.item() * random_input[0, 2, 0].item()
u_h2 = w_hh.item() * h2.item()
print(f"  {Colors.GREEN}คำนวณแบบ manual:{Colors.ENDC}")
print(f"  W·x_3 = {w_ih.item():.4f} × {random_input[0, 2, 0].item()} = {w_x3:.4f}")
print(f"  U·h_2 = {w_hh.item():.4f} × {h2.item()} = {u_h2:.4f}")
print(f"  {Colors.GREEN}h_3 = tanh(W·x_3 + U·h_2) = tanh({w_x3:.4f} + {u_h2:.4f}) = tanh({w_x3 + u_h2:.4f}) = {h3.item():.4f}{Colors.ENDC}")

# Verify การคำนวณ
diff3 = abs(h3.item() - torch.tanh(torch.tensor(w_x3 + u_h2)).item())
if diff3 < 1e-6:
    print(f"  ✓ การ verify สำเร็จ! ความแตกต่าง: {diff3:.10f}")
else:
    print(f"  {Colors.RED}✗ การ verify ล้มเหลว! ความแตกต่าง: {diff3:.10f}{Colors.ENDC}")

print(f"\n{Colors.BOLD}=== สรุปผลการทดลองและการ Verify ==={Colors.ENDC}")
print("1. เราพบว่า RNN Cell ประกอบด้วย weights 2 ชุด:")
print(f"   - Weight W (input-to-hidden): {w_ih.item():.4f} - ใช้คำนวณผลกระทบของ input ปัจจุบัน")
print(f"   - Weight U (hidden-to-hidden): {w_hh.item():.4f} - ใช้คำนวณผลกระทบของ hidden state ก่อนหน้า")
print("\n2. RNN ใช้ weights เดียวกันในทุก time step (weight sharing)")
print("   - พิสูจน์ได้จากการคำนวณ h_1, h_2, และ h_3 โดยใช้ W และ U ค่าเดียวกัน")
print("   - นี่เป็นคุณสมบัติสำคัญที่ทำให้ RNN สามารถรับข้อมูลที่มีความยาวไม่แน่นอนได้")
print("\n3. ข้อมูลจากอดีตมีผลต่อการทำนายในปัจจุบัน")
print("   - h_0 มีผลต่อ h_1 ผ่านการคูณกับ U")
print("   - h_1 มีผลต่อ h_2 ผ่านการคูณกับ U")
print("   - h_2 มีผลต่อ h_3 ผ่านการคูณกับ U")
print("   - นี่คือกลไกที่ทำให้ RNN มี 'ความจำ' และเหมาะกับข้อมูลแบบลำดับ (sequence data)")
print(f"\n4. {Colors.BOLD}การ Verify แสดงให้เห็นว่า:{Colors.ENDC}")
print(f"   - {Colors.BLUE}PyTorch RNN Cell{Colors.ENDC} และ {Colors.GREEN}การคำนวณด้วยมือ{Colors.ENDC} ให้ผลลัพธ์ที่เหมือนกัน")
print("   - ยืนยันความถูกต้องของสมการ h_t = tanh(W·x_t + U·h_{t-1}) ในการอธิบายการทำงานของ RNN Cell")
print(f"\n{Colors.BOLD}หมายเหตุ:{Colors.ENDC} การใช้สีในการแสดงผลช่วยแยกผลลัพธ์จากแต่ละวิธีให้เห็นได้ชัดเจนขึ้น")
print(f"  - {Colors.BLUE}สีฟ้า{Colors.ENDC}: ผลลัพธ์จาก PyTorch RNN Cell โดยตรง")
print(f"  - {Colors.GREEN}สีเขียว{Colors.ENDC}: ผลลัพธ์จากการคำนวณด้วยมือตามสมการ RNN")

Parameters ของ RNN Cell:
Weight W (input-to-hidden): 0.3116 - ใช้คำนวณอิทธิพลของ input ปัจจุบัน
Weight U (hidden-to-hidden): -0.3960 - ใช้คำนวณอิทธิพลของ hidden state ก่อนหน้า

สมการของ RNN Cell: h_t = tanh(W·x_t + U·h_{t-1})
  โดย:
    - h_t คือ hidden state ณ เวลา t
    - x_t คือ input ณ เวลา t
    - h_{t-1} คือ hidden state ก่อนหน้า
    - W และ U คือ weight matrices ที่ใช้ในทุก time step (weight sharing)

Input sequence:
X_1 = 1.0
X_2 = -0.7774999737739563
X_3 = 0.2587999999523163

Initial hidden state (h_0): 0.0

การคำนวณ hidden state ที่แต่ละ time step:

Time step 1:
  PyTorch RNN Cell → h_1 = 0.3019  (คำนวณโดย RNN Cell ของ PyTorch)
  คำนวณแบบ manual:
  W·x_1 = 0.3116 × 1.0 = 0.3116
  U·h_0 = -0.3960 × 0.0 = -0.0000
  h_1 = tanh(W·x_1 + U·h_0) = tanh(0.3116 + -0.0000) = tanh(0.3116) = 0.3019
  ✓ การ verify สำเร็จ! ความแตกต่าง: 0.0000000000

Time step 2:
  PyTorch RNN Cell → h_2 = -0.3468  (คำนวณโดย RNN Cell ของ PyTorch)
  คำนวณแบบ manual:
  W·x_2 = 0.3116 × -0.7774999737739563 = -